In [ ]:
from abc import ABC, abstractmethod

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
file = open("dane/dane{}.txt".format(1), "r")
X = []
y = []
for x in file:
  point = x.split(" ")
  point.remove('\n')
  X.append(float(point[0]))
  y.append(float(point[1]))

#norm=np.linalg.norm(y) #opcjonalnie
#y/=norm #opcjonalnie
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
class ActivationFunction(ABC):
  @abstractmethod
  def derivative(self, x):
    pass
  @abstractmethod
  def __call__(self, x):
    pass

In [ ]:
class Sigmoid(ActivationFunction):
  def __call__(self, x):
    return 1 / (1 + np.exp(-x))
  def derivative(self, x):
    a = self(x)
    return a * (1 - a)

In [ ]:
class Relu(ActivationFunction):
  def __call__(self, x):
    return np.max(0, x)
  def derivative(self, x):
    Sigmoid()(x)

In [ ]:
class Neuron:
  def __init__(self, input: int, f: ActivationFunction, eta: float):
    self.W = np.random.rand(input)
    self.Wb = np.random.rand(1)[0]
    self.f = f
    self.eta = eta
    self.s = 0

  def predict(self, x):
    self.s = np.dot(x, self.W) + self.Wb
    y = self.f(self.s)
    return y, self.s

  def fit(self, e, s):
    d = self.f.derivative(s) * e
    self.W += self.eta * d * self.W * e
    return 1.

In [ ]:
# class Layer:
#   def __init__(self, input, output, f, eta):
#     self.neurons = []
#     for i in range(output):
#       self.neurons.append(Neuron(input, f, eta))
#
#   def predict(self, x):
#     output = np.zeros(len(self.neurons))
#     s = []
#     for i, n in enumerate(self.neurons):
#       output[i], s = n.predict(x)
#     return output, s
#
#   def fit(self, e, s):
#     #Zapisac wagi dla wyliczenia bledu do poprzedniej warstwy
#     #Wykonac korekcje wag dla aktualnej warstwy
#     return 1.
#
#   def getW(self):
#     allW = []
#     for n in self.neurons:
#       for w in n.W:
#         allW.append(w)
#     return np.array(allW)

class Layer:
  def __init__(self, input, output, f: ActivationFunction, eta):
    self.weights = np.random.rand(input, output)
    self.bias = np.random.rand(input)
    self.f = f
    self.alpha = eta

  def predict(self, x):
    self.z = self.weights @ x + self.bias
    self.a = self.f(self.z)
    return self.a

  def fit(self, e, s):
    #Zapisac wagi dla wyliczenia bledu do poprzedniej warstwy
    #Wykonac korekcje wag dla aktualnej warstwy
    return 1.

  def getW(self):
    allW = []
    for n in self.neurons:
      for w in n.W:
        allW.append(w)
    return np.array(allW)

In [ ]:
# class NeuronNetwork:
#   def __init__(self, layers, acti, eta):
#     self.layers=[]
#     for i in range(1, len(layers)):
#       self.layers.append(Layer(layers[i-1], layers[i], acti, eta))
#
#   def predict(self, x):
#     s = np.array([])
#     out = x
#     for layer in self.layers:
#       out, si = layer.predict(out)
#       s.append(si)
#     return out, s
#
#   def fit(self, X, y, e, s):
#     #Idac od warstwy wyjsciowej, pobrac wagi i blad aktualnej warstwy do wyliczenia bledu dla kolejnej, i przeprowadzic trening aktualnej warstwy.
#     return 1.

class NeuronNetwork:
  def __init__(self, layers, acti, eta):
    self.layers = []
    for i in range(1, len(layers)):
      self.layers.append(Layer(layers[i-1], layers[i], acti, eta))

  def predict(self, x):
    out = x
    for layer in self.layers:
      out = layer.predict(out)
    return out

  def backpropagate(self, y):
    errors = np.zeros(len(self.layers))
    partials = np.zeros((self.layers-1, 2))
    last = self.layers[-1]
    errors[-1] = -(y - last.a) * last.f.derivative(last.z)
    for i, layer in reversed(enumerate(self.layers[:-1])):
      errors[i] = layer.weights.T * errors[i+1] * layer.f.derivative(layer.z)
      partials[i, 0], partials[i, 1] = errors[i+1] @ layer.a.T, errors[i+1]
    return errors, partials

  def fit(self, X, y, e, s):
    for
    #Idac od warstwy wyjsciowej, pobrac wagi i blad aktualnej warstwy do wyliczenia bledu dla kolejnej, i przeprowadzic trening aktualnej warstwy.
    return 1.

In [ ]:
def online(NN, epoch=100):
  for i in range(epoch):
    for x, y in zip(X_train, y_train):
      e, s = NN.predict(x)
      e -= y
      NN.fit(X_train, y_train, e, s)

In [ ]:
def batch(NN, epoch=100):
  for i in range(epoch):
    e = 0
    s = 0
    for x, y in zip(X_train, y_train):
      en, sn = NN.predict(x)
      e += (en-y)
      s += sn
    e /= len(X_train)
    s /= len(X_train)
    NN.fit(X_train, y_train, e, s)

In [ ]:
batch(NeuronNetwork([1, 10, 20, 50, 1], Sigmoid(), 0.001), 1)
online(NeuronNetwork([1, 10, 20, 50, 1], Sigmoid(), 0.001), 1)

In [ ]:
#utworzyc model, wytrenowac i przetestowac
#pomanipulowac iloscia neuronow, warstw, funkcji aktywacji, wspolczynikiem uczenia i iloscia epok